# Algoritmos Gulosos


## Huffman


O código de Huffman é um algoritmo de compressão de dados sem perdas. A ideia é associar a largura variada de códigos aos caracteres de entrada, a largura dos códigos associados é proporcional a frequência dos caracteres de entrada.

Os códigos associados são chamados de códigos de prefixo, tais são códigos binários os quais garantem que um código não seja prefixo de outro.

O tempo de complexidade do algoritmo de Huffman implementado abaixo é de O(nLogn).

1.  Crie duas filas de prioridades vazias.
2.  Crie um nó folha para cada caractere e coloque-o na primeira fila em ordem crescente de frequência. Inicialize a segunda fila como vazia.
3.  Retire os dois nós com as menores frequências examinando a frente de ambas as filas. Repetindo os seguintes passos duas vezes:

        a. Se a segunda fila está vazia, retire da primeira fila.
        b. Se a primeira fila está vazia, retire da segunda fila.
        c. Senão, compare a frente das duas filas e retire o nó mínimo.

4.  Crie um novo nó interno com a frequência igual a soma das frequêcias dos dois nós. Faça o primeiro nó retirado como o filho esquerdo e o segundo nó retirado como filho direito. Enfileire este nó na segunda fila.
5.  Repita os passos 3 e 4 enquanto houver pelo menos um nó nas filas. O nó restante é o nó raíz e completamos a árvore.


Para alcançarmos a implementaçao de Huffman criaremos uma classe para as filas de prioridades `Queue` e para os nós de Huffman `Node`:


In [201]:
# Classe para representar um nó da árvore de Huffman
class Node:

    def __init__(self, data=None, freq=None,
                 left=None, right=None):
        self.data = data
        self.freq = freq
        self.left = left
        self.right = right
        self.huff = ''

    # Função para checar se o nó é uma folha
    def isLeaf(self):
        return (self.left == None and
                self.right == None)

# Classe para representar uma fila de prioridade
class Queue:

    def __init__(self):
        self.queue = []

    # Função para checar se a fila tem tamanho 1
    def isSizeOne(self):
        return len(self.queue) == 1

    # Função para checar se a fila está vazia
    def isEmpty(self):
        return self.queue == []

    # Função para adicionar um item à fila
    def enqueue(self, x):
        self.queue.append(x)

    # Função para remover um item da fila
    def dequeue(self):
        return self.queue.pop(0)

# Função para extrair o nó de menor frequência das filas
def findMin(firstQueue, secondQueue):

    # Passo 3.1: Se a primeira fila estiver vazia,
    # remova da segunda fila
    if secondQueue.isEmpty():
        return [firstQueue.dequeue(), "first"]

    # Passo 3.2: Se a segunda fila estiver vazia,
    # remova da primeira fila
    if firstQueue.isEmpty():
        return [secondQueue.dequeue(), "second"]

    # Passo 3.3: Se a frequência do primeiro item da
    # primeira fila for menor que a frequência do primeiro
    # item da segunda fila, remova da primeira fila
    if (firstQueue.queue[0].freq <
            secondQueue.queue[0].freq):
        return [firstQueue.dequeue(), "first"]
    
    # Passo 3.4: Se a frequência do primeiro item da
    # segunda fila for igual que a frequência do primeiro
    # item da primeira fila, remova da primeira fila
    if (firstQueue.queue[0].freq ==
            secondQueue.queue[0].freq):
        return [firstQueue.dequeue(), "first"]


    return [secondQueue.dequeue(), "second"]


# Função para imprimir os códigos de Huffman
def printCodes(root, arr):

    if root.left:
        arr.append(0)
        printCodes(root.left, arr)
        arr.pop(-1)

    if root.right:
        arr.append(1)
        printCodes(root.right, arr)
        arr.pop(-1)

    if root.isLeaf():
        print(f"{root.data}: ", end="")
        for i in arr:
            print(i, end="")

        print()

# Função para imprimir a árvore de Huffman
def printTree(root: Node):
    def height(root: Node):
        return 1 + max(height(root.left), height(root.right)) if root else -1  
    nlevels = height(root)
    width =  pow(2,nlevels+1)

    q=[(root,0,width,'c')]
    levels=[]

    while(q):
        node,level,x,align= q.pop(0)
        if node:            
            if len(levels)<=level:
                levels.append([])
        
            levels[level].append([node,level,x,align])
            seg= width//(pow(2,level+1))
            q.append((node.left,level+1,x-seg,'l'))
            q.append((node.right,level+1,x+seg,'r'))

    for i,l in enumerate(levels):
        pre=0
        preline=0
        linestr=''
        pstr=''
        seg= width//(pow(2,i+1))
        for n in l:
            valstr=''
            if n[0].data=="$":
              valstr=str(n[0].freq)
            else:
              valstr=str(n[0].data)
            if n[3]=='r':
                linestr+=' '*(n[2]-preline-1-seg-seg//2)+ '¯'*(seg +seg//2)+'\\'
                preline = n[2] 
            if n[3]=='l':
               linestr+=' '*(n[2]-preline-1)+'/' + '¯'*(seg+seg//2)  
               preline = n[2] + seg + seg//2
            pstr+=' '*(n[2]-pre-len(valstr))+valstr #correct the potition acording to the number size
            pre = n[2]
        print(linestr)
        print(pstr)  


Com isso, chegamos na função `Huffman` que monta a árvore de Huffman.

In [202]:
# Constroi a arvore de Huffman e retorna a raiz
# da arvore e imprime os codigos de Huffman de modo
# transversal
def Huffman(data, freq, size):

    # Passo 1: Crie uma fila vazia
    firstQueue = Queue()
    secondQueue = Queue()

    k = 0
    print("-------------------------")
    print(f"i = {k}")
    print("")

    # Passo 2: Crie um nó para cada símbolo
    # e enfileire cada nó na primeira fila
    for i in range(size):
        firstQueue.enqueue(Node(data[i], freq[i]))
        print(f"{data[i]}", end='    ')

    print("")

    # Roda até que haja apenas um nó na segunda fila e
    # todos os nós tenham sido removidos da primeira fila
    # pois o ultimo nó da segunda fila será a raiz da arvore
    while not (firstQueue.isEmpty() and
               secondQueue.isSizeOne()):
        
        k += 1
        # Passo 3: Retire os dois nós de menor
        # frequência da fila 1 e 2 e crie um
        # novo nó interno com a soma das duas
        # frequências. Enfileire este nó na fila 2.
        left, queue1 = findMin(firstQueue, secondQueue)
        right, queue2 = findMin(firstQueue, secondQueue)

        if queue1 == queue2:
          if right.freq == left.freq:
            aux = right
            right = left
            left = aux

        left.huff = 0
        right.huff = 1

        # Passo 4: Crie um novo nó interno com
        # frequência igual à soma das frequências
        # dos dois nós de menor frequência
        # removidos.
        top = Node("$", left.freq + right.freq,
                   left, right)
        print("-------------------------")
        print(f"i = {k}")
        print(f"        {top.freq}       ")
        print(f"       /  \\       ")
        print(f"      {left.freq if left.data == '$' else left.data}   {right.freq if right.data == '$' else right.data}      ")
        secondQueue.enqueue(top)

    print("-------------------------")
    root = secondQueue.dequeue()

    # Impressão dos códigos e árvore de Huffman
    arr = []
    printTree(root)
    printCodes(root, arr)

    return root


In [203]:
arr = ["e", "r", "s", "t", "n", "l", "z", "x"]
freq = [34, 22, 24, 28, 15, 10, 9 , 8]
size = len(arr)

Huffman(arr, freq, size)


-------------------------
i = 0

e    r    s    t    n    l    z    x    
-------------------------
i = 1
        56       
       /  \       
      e   r      
-------------------------
i = 2
        52       
       /  \       
      s   t      
-------------------------
i = 3
        25       
       /  \       
      n   l      
-------------------------
i = 4
        17       
       /  \       
      z   x      
-------------------------
i = 5
        108       
       /  \       
      56   52      
-------------------------
i = 6
        42       
       /  \       
      25   17      
-------------------------
i = 7
        150       
       /  \       
      108   42      
-------------------------

             150
       /¯¯¯¯¯¯   ¯¯¯¯¯¯\
     108              42
   /¯¯¯ ¯¯¯\       /¯¯¯ ¯¯¯\
  56      52      25      17
 /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\
 e   r   s   t   n   l   z   x
e: 000
r: 001
s: 010
t: 011
n: 100
l: 101
z: 110
x: 111


In [204]:
# -------------------------
# i = 0

# e    r    s    t    n    l    z    x    
# -------------------------
# i = 1
#         56       
#        /  \       
#       e   r      
# -------------------------
# i = 2
#         52       
#        /  \       
#       s   t      
# -------------------------
# i = 3
#         25       
#        /  \       
#       n   l      
# -------------------------
# i = 4
#         17       
#        /  \       
#       z   x      
# -------------------------
# i = 5
#         108       
#        /  \       
#       56   52      
# -------------------------
# i = 6
#         42       
#        /  \       
#       25   17      
# -------------------------
# i = 7
#         150       
#        /  \       
#       108   42      
# -------------------------

#              150
#        /¯¯¯¯¯¯   ¯¯¯¯¯¯\
#      108              42
#    /¯¯¯ ¯¯¯\       /¯¯¯ ¯¯¯\
#   56      52      25      17
#  /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\
#  e   r   s   t   n   l   z   x

# e: 000
# r: 001
# s: 010
# t: 011
# n: 100
# l: 101
# z: 110
# x: 111


In [205]:
arr = ["a", "b", "c", "d", "e", "f", "g", "h"]
freq = [1, 1, 2, 3, 5, 8, 13, 21]
size = len(arr)

Huffman(arr, freq, size)

-------------------------
i = 0

a    b    c    d    e    f    g    h    
-------------------------
i = 1
        2       
       /  \       
      b   a      
-------------------------
i = 2
        4       
       /  \       
      c   2      
-------------------------
i = 3
        7       
       /  \       
      d   4      
-------------------------
i = 4
        12       
       /  \       
      e   7      
-------------------------
i = 5
        20       
       /  \       
      f   12      
-------------------------
i = 6
        33       
       /  \       
      g   20      
-------------------------
i = 7
        54       
       /  \       
      h   33      
-------------------------

                                                                                                                                                                                                                                                              54
                                 

In [206]:
#     54
# /¯¯    ¯¯ \
# h         33
#       /¯¯¯  ¯¯¯\
#       g        20
#           /¯¯¯  ¯¯¯\
#           f        12
#                 /¯¯¯  ¯¯¯\
#                 e        7
#                     /¯¯¯   ¯¯¯\
#                     d          4
#                             /¯¯¯ ¯¯¯\
#                             c       2
#                                   /¯ ¯\
#                                   b   a
# h: 0
# g: 10
# f: 110
# e: 1110
# d: 11110
# c: 111110
# b: 1111110
# a: 1111111

# fdheg: 110111100111010